In [186]:
import imp
import pandas as pd

from joblib import dump, load

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, mean_absolute_error
from sklearn.model_selection import train_test_split

import scipy.stats as stats

import json

In [187]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

## 1. Carga de los datos

In [188]:
df = pd.read_csv('./data/university_admission_train.csv')

In [189]:
df.shape

(1569, 10)

In [190]:
df.head(5)

,Unnamed: 0,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Admission Points
0,0,479,327,113,4,4.00,2.77,8.88,1,84.47
1,1,446,301,92,1,1.85,1.50,7.71,0,45.08
2,2,336,297,100,1,2.41,1.59,7.89,0,47.42
3,3,20,303,98,3,3.50,3.00,8.50,0,62.00
4,4,432,320,94,2,1.38,3.50,8.78,1,73.00


Al hacer un .head() podemos evidenciar que los datos contienen únicamente valores numéricos que corresponden generalmente a valores de verdad o puntajes en rangos determinados, en el diccionario de datos brindado.

### Eliminamos la columna Unnamed: 0
Esta columna no aporta nada al modelo ya que es la mala construcción de un identificador y no es influyente en la columna objetivo "Admission Points"

In [191]:
df = df.drop(columns=['Unnamed: 0'])

# Eliminamos todas las filas que tengan algun valor nulo

In [192]:
# Se eliminan las filas que tengan valores nulos en la columna Admission Points debido a que esa es la 
# variable objetivo y necesitamos predecirla. 
df = df.dropna()

# Dividimos los datos en train y en test

In [193]:
X_train, X_test, Y_train, Y_test = train_test_split(df, df["Admission Points"], test_size=0.2, random_state=1)

## Entrenamiento para un primera modelo de regresion

## Creamos el ColumnTransformer
Aqui se normalizamos entre 0 y 1 las columnas que vamos a utilizar y se eliminan las columnas que no necesitamos.

In [194]:
ct = ColumnTransformer(
    [("gre_preprocess", MinMaxScaler(), ["GRE Score", "University Rating", "SOP", "CGPA", "Research"]),
    ("drop_columns", "drop", ['Serial No.', "TOEFL Score", "LOR "])])

### Creamos el PipeLine 
Creamos el Pipeline con el ColumnTransformer que creamos arriba y con un modelo de regresion lineal

In [195]:
pipeline = Pipeline(
    [
        ('feature_selection', ct),
        ('model', LinearRegression())
    ]
)

In [196]:
pipeline = pipeline.fit(X_train, Y_train)

## 4. Analisis de coeficientes

In [197]:
preds_train = pipeline.predict(X_train)
preds_test = pipeline.predict(X_test)

In [198]:
pipeline.score(X_train, Y_train)

0.70333362037035

In [199]:
pipeline.score(X_test, Y_test)

0.7017157637189441

In [200]:
mae_error_train = mean_absolute_error(Y_train, preds_train)
print("MAE ERRROR TRAIN: ", mae_error_train)

MAE ERRROR TRAIN:  7.9474396268892535


In [201]:
mae_error_test = mean_absolute_error(Y_test, preds_test)
print("MAE ERRROR TEST: ", mae_error_test)

MAE ERRROR TEST:  7.3845448676606535


## 5. Persistencia modelo

Definición del $PATH donde se va a persistir el .joblib

In [202]:
filename = './assets/modelo.joblib'

Persistencia del .joblib con el pipeline

In [203]:
dump(pipeline, filename) 

['./assets/modelo.joblib']

Prueba del pipeline mediante la carga del .joblib

In [204]:
pipeline2 = load(filename)

Ejecución del modelo mediante el pipeline con los datos previamente spliteados

In [205]:
pipeline2.score(X_test, Y_test)

0.7017157637189441

Generación de las predicciones con el dataset de pruebas

In [206]:
df2 = pd.read_csv('./data/university_admission_test.csv')

In [207]:
p = pipeline2.predict(df2)